# Import Libraries

In [12]:
import pandas as pd
import requests as req
import json
from datetime import datetime, timedelta

pd.options.display.max_rows = 400

# Retrieve RW Metadata

In [7]:
# Base URL for getting dataset metadata from RW API
# Metadata = Data that describes Data 
url = "https://api.resourcewatch.org/v1/dataset?sort=slug,-provider,userId&status=saved&includes=metadata,vocabulary,widget,layer"

# page[size] tells the API the maximum number of results to send back
# There are currently between 200 and 300 datasets on the RW API
payload = { "application":"rw", "page[size]": 1000}

# Request all datasets, and extract the data from the response
res = req.get(url, params=payload)
data = res.json()["data"]

#############################################################

### Convert the json object returned by the API into a pandas DataFrame
# Another option: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.io.json.json_normalize.html
datasets_on_api = {}
for ix, dset in enumerate(data):
    atts = dset["attributes"]
    metadata = atts["metadata"]
    layers = atts["layer"]
    widgets = atts["widget"]
    tags = atts["vocabulary"]
    datasets_on_api[atts["name"]] = {
        "rw_id":dset["id"],
        "table_name":atts["tableName"],
        "provider":atts["provider"],
        "date_updated":atts["updatedAt"],
        "num_metadata":len(metadata),
        "metadata": metadata,
        "num_layers":len(layers),
        "layers": layers,
        "num_widgets":len(widgets),
        "widgets": widgets,
        "num_tags":len(tags),
        "tags":tags
    }

# Create the DataFrame, name the index, and sort by date_updated
# More recently updated datasets at the top
current_datasets_on_api = pd.DataFrame.from_dict(datasets_on_api, orient='index')
current_datasets_on_api.index.rename("Dataset", inplace=True)
current_datasets_on_api.sort_values(by=["date_updated"], inplace=True, ascending = False)

# Import, Format, Export data

## Connecting refugee data with wri_bounds

In [4]:
refugees = pd.read_excel('Refugee_CSV.xlsx').set_index('Country')
bounds = req.get('https://raw.githubusercontent.com/wri/wri-bounds/master/dist/all_primary_countries.geojson').json()

In [33]:
refugees

,Refugees
Country,
Afghanistan,13969.0
Albania,41333.0
Algeria,2509.0
Andorra,38.0
Angola,1663.0
Antigua and Barbuda,19.0
Argentina,1082.0
Armenia,43432.0
Australia,9.0


In [39]:
for ix, cntry in enumerate(bounds['features']):
    name = cntry['properties']['name']
    if name not in refugees.index:
        print(name)
        new_name = input('What should new name be?')
        if new_name: 
            bounds['features'][ix]['properties']['name'] = new_name
            bounds['features'][ix]['properties']['refugee_count'] = refugees.loc[new_name, 'Refugees']
        else:
            print('no data')
            bounds['features'][ix]['properties']['refugee_count'] = None
    else:
        bounds['features'][ix]['properties']['refugee_count'] = refugees.loc[name, 'Refugees']

Kiribati
What should new name be?
no data
South Korea
What should new name be?Rep. of Korea
Laos
What should new name be?Lao People's Dem. Rep.
Liechtenstein
What should new name be?
no data
Luxembourg
What should new name be?
no data
Moldova
What should new name be?Rep. of Moldova
Macedonia
What should new name be?The former Yugoslav Republic of Macedonia
North Korea
What should new name be?Dem. People's Rep. of Korea
Russia
What should new name be?Russian Federation
Solomon Islands
What should new name be?
no data
Republic of Serbia
What should new name be?Serbia and Kosovo (S/RES/1244 (1999))
Sao Tome and Principe
What should new name be?
no data
Syria
What should new name be?Syrian Arab Rep.
Timor-Leste
What should new name be?
no data
Tuvalu
What should new name be?
no data
United Republic of Tanzania
What should new name be?United Rep. of Tanzania
United States of America
What should new name be?
no data
Venezuela
What should new name be?Venezuela (Bolivarian Republic of)
Vietnam

In [45]:
json.dump(bounds, open('bounds_with_refugee_counts.geojson','w'))

## Creating data frame for scatter plots

In [85]:
refugees = refugees.reset_index()
refugees

,Country,Refugees
0,Afghanistan,13969.0
1,Albania,41333.0
2,Algeria,2509.0
3,Andorra,38.0
4,Angola,1663.0
5,Antigua and Barbuda,19.0
6,Argentina,1082.0
7,Armenia,43432.0
8,Australia,9.0
9,Austria,37.0


### Water Stress

In [22]:
query_base = "https://api.resourcewatch.org/v1/query/{}?sql={}"

sql = "SELECT * FROM {}"
DT_FORMAT = '%Y-%m-%dT%H:%M:%SZ'

today = datetime.today()
five_years_ago = today - timedelta(days=360*5)
cutoff = datetime.strftime(five_years_ago, DT_FORMAT)

#[ds for ds in current_datasets_on_api.index if 'stress' in ds.lower()]

ds = [ds for ds in current_datasets_on_api.index if 'stress' in ds.lower()][0]
ds_id = current_datasets_on_api.loc[ds, 'rw_id']
table_name = current_datasets_on_api.loc[ds, 'table_name']

#query_sql = sql.format(table_name, cutoff)
query_sql = sql.format(table_name)
query = query_base.format(ds_id, query_sql) 
res = req.get(query).json()
water_stress = pd.DataFrame(res['data'])

In [52]:
water_stress = water_stress[['all_sectors', 'name']]

### GDP

In [39]:
query_base = "https://api.resourcewatch.org/v1/query/{}?sql={}"

sql = "SELECT * FROM {} WHERE year = 2015"
DT_FORMAT = '%Y-%m-%dT%H:%M:%SZ'

#[ds for ds in current_datasets_on_api.index if 'gross' in ds.lower()]

ds = [ds for ds in current_datasets_on_api.index if 'gross' in ds.lower()][0]
ds_id = current_datasets_on_api.loc[ds, 'rw_id']
table_name = current_datasets_on_api.loc[ds, 'table_name']

query_sql = sql.format(table_name)
query = query_base.format(ds_id, query_sql) 
res = req.get(query).json()

gdp = pd.DataFrame(res['data'])

In [54]:
gdp = gdp[['rw_country_code', 'rw_country_name', 'yr_data']]

### Political Freedoms Index

In [47]:
query_base = "https://api.resourcewatch.org/v1/query/{}?sql={}"

sql = "SELECT * FROM {} "
DT_FORMAT = '%Y-%m-%dT%H:%M:%SZ'

#[ds for ds in current_datasets_on_api.index if 'political' in ds.lower()]

ds = [ds for ds in current_datasets_on_api.index if 'political' in ds.lower()][1]
ds_id = current_datasets_on_api.loc[ds, 'rw_id']
table_name = current_datasets_on_api.loc[ds, 'table_name']

query_sql = sql.format(table_name)
query = query_base.format(ds_id, query_sql) 
res = req.get(query).json()

political_freedoms = pd.DataFrame(res['data'])

In [55]:
political_freedoms = political_freedoms[['rw_country_code', 'rw_country_name', 'total_aggr']]

## Joined

In [59]:
tmp = political_freedoms.merge(gdp, left_on='rw_country_code', right_on='rw_country_code')

In [65]:
aliases = tmp[['rw_country_code', 'rw_country_name_x']]
aliases.columns = ['code', 'name']

In [68]:
water_stress

,all_sectors,name
0,5.000000,Antigua and Barbuda
1,5.000000,Bahrain
2,5.000000,Barbados
3,5.000000,Comoros
4,5.000000,Cyprus
5,5.000000,Dominica
6,5.000000,Jamaica
7,5.000000,Malta
8,5.000000,Qatar
9,5.000000,Saint Lucia


In [80]:
def find_rw_country_code(name, aliases):
    if name in aliases['name'].values:
        code = aliases.loc[aliases['name'].isin([name]), 'code'].values[0]
        return code
    else:
        print(name)
        code = input("Country Code?")
        if code:
            return code
        else:
            return None
    
water_stress['rw_country_code'] = water_stress.apply(lambda row: find_rw_country_code(row['name'], aliases), axis = 1)
water_stress

Western Sahara
Country Code?ESH
East Timor
Country Code?TLS
Palestine
Country Code?PSE
Somaliland
Country Code?
Vatican
Country Code?VAT
Kosovo
Country Code?
Romania
Country Code?ROU
Republic of the Congo
Country Code?COG
Taiwan
Country Code?TWN
Curacao
Country Code?


/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,all_sectors,name,rw_country_code
0,5.000000,Antigua and Barbuda,ATG
1,5.000000,Bahrain,BHR
2,5.000000,Barbados,BRB
3,5.000000,Comoros,COM
4,5.000000,Cyprus,CYP
5,5.000000,Dominica,DMA
6,5.000000,Jamaica,JAM
7,5.000000,Malta,MLT
8,5.000000,Qatar,QAT
9,5.000000,Saint Lucia,LCA


In [81]:
tmp = tmp.merge(water_stress, left_on='rw_country_code', right_on='rw_country_code')
tmp

,rw_country_code,rw_country_name_x,total_aggr,rw_country_name_y,yr_data,all_sectors,name
0,THA,Thailand,32,Thailand,4.013994e+11,1.698024,Thailand
1,TJK,Tajikistan,11,Tajikistan,7.853450e+09,3.528026,Tajikistan
2,ARG,Argentina,82,Argentina,5.947493e+11,2.513095,Argentina
3,MRT,Mauritania,30,Mauritania,4.844223e+09,0.604109,Mauritania
4,MCO,Monaco,84,Monaco,NaN,2.664468,Monaco
5,ZWE,Zimbabwe,32,Zimbabwe,1.630467e+10,0.640418,Zimbabwe
6,YEM,Yemen,14,Yemen,3.460248e+10,4.674046,Yemen
7,BHR,Bahrain,12,Bahrain,3.112585e+10,5.000000,Bahrain
8,VNM,Vietnam,20,Vietnam,1.932411e+11,1.008669,Vietnam
9,VEN,Venezuela,30,Venezuela,NaN,2.299918,Venezuela


In [86]:
refugees['rw_country_code'] = refugees.apply(lambda row: find_rw_country_code(row['Country'], aliases), axis = 1)

Bahamas
Country Code?BHS
Cayman Islands
Country Code?CYM
Congo
Country Code?COG
Côte d'Ivoire
Country Code?CIV
Dem. People's Rep. of Korea
Country Code?PRK
Dem. Rep. of the Congo
Country Code?COD
Dominican Rep.
Country Code?DOM
French Guiana
Country Code?GUF
Gibraltar
Country Code?GIB
Guadeloupe
Country Code?GLP
Guinea-Bissau
Country Code?GNB
Iran (Islamic Rep. of)
Country Code?IRN
Lao People's Dem. Rep.
Country Code?LAO
Micronesia (Federated States of)
Country Code?FSM
New Caledonia
Country Code?NCL
Niue
Country Code?NIU
Palestinian
Country Code?PSE
Rep. of Korea
Country Code?KOR
Rep. of Moldova
Country Code?MDA
Romania
Country Code?ROU
Russian Federation
Country Code?RUS
Serbia and Kosovo (S/RES/1244 (1999))
Country Code?SRB
Stateless
Country Code?
Syrian Arab Rep.
Country Code?SYR
The former Yugoslav Republic of Macedonia
Country Code?MKD
Turks and Caicos Islands
Country Code?TCA
United Rep. of Tanzania
Country Code?TZA
Various/Unknown
Country Code?
Venezuela (Bolivarian Republic of

In [87]:
tmp = tmp.merge(refugees, left_on='rw_country_code', right_on='rw_country_code')


In [90]:
tmp = tmp.drop(['rw_country_name_y', 'name', 'Country'], axis=1)

In [93]:
final_data = tmp
final_data.columns = ['Country Code', 'Country Name', 'Political Freedom Index', 'GDP', 'Water Stress, all Sectors', 'Refugees to US in Past 10 Years']
final_data.to_csv('scatterplotdata.csv')